In [ ]:
import os
from dataclasses import dataclass
import logging
logging.basicConfig(level=logging.INFO, 
    format='%(asctime)s %(levelname)s - %(message)s',
    datefmt="%H:%M:%S"
)

import numpy as np
%rm -rf my-experiment*
%rm -rf ioh_data*

In [ ]:
import ioh
%pip show ioh

# Problem
In ioh, everything revolves around the `problem` class, which exists for both `Real` and `Integer` types for continious and discrete problems resp. These classes are wrappers around an objective function, and can be used to interact with the various other parts of iohexperimenter. 

We have a number of objective functions already implemented for convenience, which includes the objective functions from the BBOB single objective benchmark by the COCO platform for the continous case and the PBO benchmark functions for the discrete case.


In [ ]:
# A list of problems can be accessed via the base classes
ioh.problem.Real.problems

In [ ]:
# In order to instantiate a problem instance, we can do the following:
problem = ioh.get_problem(
    "Sphere", 
    instance=1,
    dimension=10,
    problem_type="Real"
)
problem

In [ ]:
# The problem class includes information about the problem, which can be retrieved via the meta_data accessor
problem.meta_data.name

In [ ]:
# The current state of the problem, e.g. the number of evaluations, best seen points etc. are stored in the problems state.
problem.state

In [ ]:
# Every problem as has a simple box-bounds associcated
problem.bounds

In [ ]:
# We can access the contraint information of the problem
x0 = np.random.uniform(problem.bounds.lb, problem.bounds.ub)

# Evaluation happens like a 'normal' objective function would
problem(x0)

# Whenever the problem is evaluated, the state changes
problem.state

In [ ]:
# If we want to perform multiple runs with the same objective function, after every run, the problem has to be reset, 
# such that the internal state reflects the current run.
def run_experiment(problem, algorithm, n_runs=5):
    for run in range(n_runs):
        
        # Run the algorithm on the problem
        algorithm(problem)

        # print the best found for this run
        print(f"run: {run+1} - best found:{problem.state.current_best.y: .3f}")

        # Reset the problem
        problem.reset()

In [ ]:
@dataclass
class RandomSearch:
    'Simple random search algorithm'
    
    n: int
    length: float = 0.0
        
    def __call__(self, problem: ioh.problem.Real) -> None:
        'Evaluate the problem n times with a randomly generated solution'
        
        for _ in range(self.n):
            # We can use the problems bounds accessor to get information about the problem bounds
            x = np.random.uniform(problem.bounds.lb, problem.bounds.ub)
            self.length = np.linalg.norm(x)
            
            problem(x)  
            

In [ ]:
# using the random search algorithm, we can then run a simple experiment
run_experiment(problem, RandomSearch(10))

# Adding custom problems
As the list of already implemented problems might not contains the objective function you would like to analyze and benchmark, we include an easy to use interface to wrap your benchmark function into an iohexperimenter `problem` object. 

Currently, only single objective functions are supported, so the only requirement on the function is its signature, which should take a single array parameter, and return a single floating point number $f(\mathbf{x}) \mapsto \mathbb{R}$.

In this example we will add the function (Styblinski–Tang):

$f(\mathbf{x}) = \frac{\sum_{i=1}^n x_i^4 - 16x_i^2 + 5x_i}{2}$

defined on $[-5, 5]$

In [ ]:
def styblinski_tang(x: np.ndarray) -> float:
    return np.sum(np.power(x, 4) - (16 * np.power(x, 2)) + (5 * x)) / 2


styblinski_tang(np.array([-2.903534]*10)) # global minima

In [ ]:
# we can wrap this function in ioh, as this is is a continous function, we use wrap_real_problem:
ioh.problem.wrap_real_problem(
    styblinski_tang,                                     # Handle to the function
    name="StyblinskiTang",                               # Name to be used when instantiating
    optimization_type=ioh.OptimizationType.MIN, # Specify that we want to minimize
    lb=-5,                                               # The lower bound
    ub=5,                                                # The upper bound
)

In [ ]:
# We can create an instance of this problem, wrapped in ioh
problem = ioh.get_problem("StyblinskiTang", dimension=10)
problem

In [ ]:
problem(np.array([-2.903534]*10))

## Specifying instances
When doing bechmarking of custom problems, it can often be usefull to look at different instances of the same problem, but that perform some transformation the parameter or objective space, to test if an algorithm is invariant to such transformations. 

Here we add the following transformations to the aforementioned objective function:

$T_x(\mathbf{x}, c) = \mathbf{x} + c$

$T_y(y, c) = y * c$

We can do this by providing transformation functions to the `wrap_problem` interface. Note that these take two parameters:
 1. The operand they function on, which are the variables for the variables transformation and objective function value for the objective transformation. 
 2. An integer identifier of the instance id which is currently used. This allows you to specify alternate behavior for different instances.  

In [ ]:
# Variables transformation R^d -> R^d
def transform_variables(x: np.ndarray, instance_id:int) -> np.ndarray:
    c = (instance_id - 1) * .5
    return x + c

# Objective transformation R -> R
def transform_objectives(y: float, instance_id:int) -> float:
    c = instance_id
    return y * c

# Note that we can overwrite a previously defined problem by calling wrap_real_problem again with the same name
ioh.problem.wrap_real_problem(
    styblinski_tang,                                     
    name="StyblinskiTang",                               
    optimization_type=ioh.OptimizationType.MIN, 
    lb=-5,                                               
    ub=5,      
    
    # Adding the transformation functions
    transform_variables=transform_variables,     
    transform_objectives=transform_objectives
)

In [ ]:
# We can now create different instances of the same problem
instance1 = ioh.get_problem("StyblinskiTang", instance=1, dimension=10)
instance2 = ioh.get_problem("StyblinskiTang", instance=2, dimension=10)
instance1, instance2

In [ ]:
# Note that when evaluating with the same point, each instance gives a different (transformed value)
instance1(x0), instance2(x0)

# Logging data
The default usage of IOHExperimenter is in generating logs of benchmarking experiments which can be analyzed in IOHAnalyzer. 

In [ ]:
import os

logger = ioh.logger.Analyzer(
    root=os.getcwd(),                  # Store data in the current working directory
    folder_name="my-experiment",       # in a folder named: 'my-experiment'
    algorithm_name="random-search",    # meta-data for the algorithm used to generate these results
    store_positions=True               # store x-variables in the logged files
)

# this automatically creates a folder 'my-experiment' in the current working directory
# if the folder already exists, it will given an additional number to make the name unique
logger

In [ ]:
%ls

In [ ]:
# In order to log data for a problem, we only have to attach it to a logger
problem = ioh.get_problem("StyblinskiTang", instance=1, dimension=2)
problem.attach_logger(logger)

# We can then run the random search as before, only now all data will be logged to a file
run_experiment(problem, RandomSearch(10), n_runs=1)

In [ ]:
%cat my-experiment/IOHprofiler_f25_StyblinskiTang.info

In [ ]:
%cat my-experiment/data_f25_StyblinskiTang/IOHprofiler_f25_DIM2.dat

## Triggers 
The default behavior of the `Analyzer` logger is to log data only when there is an improvement of the objective value. We can change this behaviour, by specifying one or more triggers, which are logical operators, which when one of them evaluates to True, will cause data to be logged.

We provide a number of trigger variants which can be use to customize the logging. In the following example, a trigger is defined which evaluates to True, every 3 function evaluations. It is combined with a trigger for improvement, so data will be logged on every 3rd function evaluation, or when there is an observed improvement of the objective value

In [ ]:
triggers = [
    ioh.logger.trigger.Each(3),
    ioh.logger.trigger.OnImprovement()
]

logger = ioh.logger.Analyzer(
    root=os.getcwd(),                  
    folder_name="my-experiment",       
    algorithm_name="random-search",    
    store_positions=True,
    
    # Add the triggers to the logger
    triggers = triggers
)

logger

In [ ]:
# Rerun the same experiment as before
problem = ioh.get_problem("StyblinskiTang", instance=1, dimension=2)
problem.attach_logger(logger)
run_experiment(problem, RandomSearch(10), n_runs=1)

In [ ]:
# We can now see that data is logged either if there is improvement, or on every 3rd evaluation
%cat my-experiment-1/data_f25_StyblinskiTang/IOHprofiler_f25_DIM2.dat

## Properties
If we want to keep track of any dynamic parameters a given algorithm might have, we can use properties to log them to the output files. 

In the following example, we will track the length parameters for the RandomSearch algorithm, which is added for illustrative purpoposes, and changes for every function evaluation

In [ ]:
# RandomSearch has a length parameter, which is dynamic
algorithm = RandomSearch(10)

# Creating a new logger
logger = ioh.logger.Analyzer(
    root=os.getcwd(),                  
    folder_name="my-experiment",       
    algorithm_name="random-search",    
    store_positions=True
)

# Before we attach a problem, we tell the logger to keep track of the length parameter on algorithm
logger.watch(algorithm, "length")

# We can now again run the same experiment 
problem = ioh.get_problem("StyblinskiTang", instance=1, dimension=2)

problem.attach_logger(logger)
run_experiment(problem, algorithm, n_runs=1)

In [ ]:
# Note the additional length parameter being logged 
%cat my-experiment-2/data_f25_StyblinskiTang/IOHprofiler_f25_DIM2.dat

# Alternate logging behaviour
We provide a number of different loggers in addition to the `Analyzer` logger, which include:
 - `FlatFile` which logs data to a simple csv file
 - `Store` which keeps all of the stored data in memory
 - `EAF/EAH` which compute Empirical Attainment Function/Histogram statistics on the fly

You can define your own custom logging behavoir by inheriting from the `AbstractLogger` class. The only required part of the interface is that you override the `__call__` operator, which takes a single `ioh.LogInfo` parameter. In this method you should define your desired behavior. 

In [ ]:
# Simple logger that logs data using the python logging module whenever it is triggeredd
class MyLogger(ioh.logger.AbstractLogger):
    def __call__(self, log_info: ioh.LogInfo):
        logging.info(msg=f"triggered! y: {log_info.y}")


# The abstract logger takes two parameters, triggers and properties
mylogger = MyLogger(triggers=[ioh.logger.trigger.ALWAYS])

problem = ioh.get_problem("StyblinskiTang", instance=1, dimension=2)
problem.attach_logger(mylogger)

run_experiment(problem, RandomSearch(10), 1)

# Standardized Experimental Setup (Python only)
In Python, we provide the `Experiment` class which can be used to easily run a given algorithm over a larger number of problems. 

In [ ]:
experiment = ioh.Experiment(
    algorithm = RandomSearch(10), # An algorithm instance
    fids = [1, 25],               # the id's of the problems we want to test
    iids = [1, 10],               # the instances 
    dims = [2, 10],               # the dimensions
    reps = 3,                     # the number of runs,
    zip_output = True       
)

In [ ]:
experiment.run()

In [ ]:
%ls ioh_data

In [ ]:
%cat ioh_data/IOHprofiler_f25_StyblinskiTang.info

In [1]:
%ls 

CMakeFiles/           ioh_data/         my-experiment-2/
CMakeLists.txt        ioh_data.zip      problem_example.h
README.md             logger_example.h  run_submodular_instances.cpp
add_new_problem.cpp   main.cpp          suite_example.h
eafh.cpp              my-experiment/    tutorial.ipynb
experiment_example.h  my-experiment-1/  tutorial.py


In [ ]:
# cleanup
%rm -rf my-experiment*
%rm -rf ioh_data*